# Step 1. List All Image Paths

In [19]:
import os

def get_train_img_list(train_img_path):
    train_img_path_list = []
    
    type_path_list = map(lambda type_name: "/".join([train_img_path, type_name]), os.listdir(train_img_path))
    print("all type paths below:")
    for p_idx in xrange(len(type_path_list)):
        print "".join(
            map(str, ["type", (p_idx+1), " dir:", type_path_list[p_idx]])
        )

    for p_idx in xrange(len(type_path_list)):
        type_img_name_list = map(lambda img_name:
                                 "/".join([type_path_list[p_idx], img_name]),
                                 os.listdir(type_path_list[p_idx]))
        print "".join(
            map(str, ["type", (p_idx+1), " images number:", len(type_img_name_list)])
        )
        train_img_path_list.extend(type_img_name_list)

    for p_idx in xrange(len(type_img_name_list[:10])): 
        print "".join(
            map(str, ["sample ", p_idx+1, ":", type_img_name_list[p_idx]])
        )
        
    # filter no-image files
    train_img_path_list = filter(lambda f: f[-4:] == ".jpg", train_img_path_list)
    return train_img_path_list

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 09:39:56.283581. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [20]:
###############################
# just for test               #
###############################

train_img_path = "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224"
img_path_list = get_train_img_list(train_img_path)
print len(img_path_list)

all type paths below:
type1 dir:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_1_seg_224
type2 dir:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_2_seg_224
type3 dir:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224
type1 images number:5760
type2 images number:17384
type3 images number:9704
sample 1:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/100-resnet-152.npy
sample 2:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/100-resnet-200.npy
sample 3:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/100-resnext-50.npy
sample 4:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/100.jpg
sample 5:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/1000-resnet-152.npy
sample 6:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 09:39:56.737291. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


# Step 2. Extract Features Using Pretrained Models

## Step 2.1 Attaining label name

In [58]:
def get_synset(synset_path="../model/synset.txt"):
    #synset_path = "../model/synset.txt"
    with open(synset_path) as f:
        synsets = [l.rstrip() for l in f]
    #print synsets
    return synsets

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 22:56:12.320341. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Step 2.2 All layers of resneXt-50 and last 10 layers:

First, have a look to observe the last 10 layers of a pretrained model:

In [59]:
###############################
# just for test               #
###############################

import mxnet as mx

prefix = "../model/finetune-resnext-50-train-add-seg-224-lr-0.01"
epoch = 3
sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)

all_layers = sym.get_internals()
all_layers.list_outputs()[-10:-1]

['stage4_unit3_bn3_output',
 '_plus15_output',
 'stage4_unit3_relu_output',
 'pool1_output',
 'flatten0_output',
 'fc1_weight',
 'fc1_bias',
 'fc1_output',
 'softmax_label']

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 22:56:14.326299. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Step 2.3 Attaining feature of flatten0_output layer from symbol:

In [60]:
def get_feature_extractor(prefix, epoch):
    
    sym, arg_params, aux_params = mx.model.load_checkpoint(prefix, epoch)

    all_layers = sym.get_internals()
    flatten_output_sym = all_layers['flatten0_output']
    mod_output = mx.mod.Module(symbol=flatten_output_sym, label_names=None, context=mx.cpu())
    mod_output.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
    mod_output.set_params(arg_params, aux_params)
    
    feature_extractor = mod_output
    return feature_extractor

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 22:56:17.127679. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Step 2.4 Attaining image and make resize

In [61]:
import cv2
import numpy as np

def get_image(filename, input_shape=(224, 224)):
    img = cv2.imread(filename)  # read image in b,g,r order
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # change to r,g,b order
    img = cv2.resize(img, input_shape)  # resize to 224*224 to fit model
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)  # change to (channel, height, width)
    img = img[np.newaxis, :]  # extend to (example, channel, heigth, width)
    return img

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 22:56:21.899916. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


# Step 2.5 Feature Extraction

Extract and save features for each image of three types.

In [65]:
import mxnet as mx
from collections import namedtuple
import time, datetime

################################################
# parameter initialization                     #
################################################

####################################
# extract feature on training set  #
####################################

Batch = namedtuple('Batch', ['data'])
train_img_path = "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224"
img_path_list = get_train_img_list(train_img_path)
feats_save_path_list = map(lambda img_path: img_path.replace("jpg", "npy"), img_path_list)
model_path = "../model"
prefix_list = ["finetune-resnet-152-train-add-seg-224",\
               "finetune-resnet-200-train-add-seg-224",\
               "finetune-resnext-50-train-add-seg-224-lr-0.01"]
epoch_list = [3, 3, 3]
feats_save_prefix_list = ["resnet-152",\
                          "resnet-200",\
                          "resnext-50"]
synset_path="../model/synset.txt"
synsets = get_synset(synset_path)

################################################
# extract feature using each pretrained model  #
################################################

for m_idx in xrange(len(prefix_list)):
    prefix = "/".join([model_path, prefix_list[m_idx]])
    epoch = epoch_list[m_idx]
    feats_save_prefix = feats_save_prefix_list[m_idx]
    feature_extractor = get_feature_extractor(prefix, epoch)
    
    for img_idx in xrange(len(img_path_list)):
        time_stamp = datetime.datetime.now()
        print("{} {}th/{} img".format(time_stamp.strftime('%Y-%m-%d %H:%M:%S'), img_idx+1, len(img_path_list))),
        
        start_extract = time.time()
        
        img_path = img_path_list[img_idx]
        feats_save_path = feats_save_path_list[img_idx]
        img = get_image(img_path)
        feature_extractor.forward(Batch([mx.nd.array(img)]))
        feats = feature_extractor.get_outputs()[0].asnumpy()
        
        #####################
        # check features    #
        #####################
        #print feats.shape
        
        feats_save_path = feats_save_path.replace(".", "-"+feats_save_prefix+".")
        #####################
        # save features     #
        #####################
        #np.save(feats_save_path, feats)
        
        finish_extract = time.time()
        duration_extract = finish_extract - start_extract
        
        print("calc and save feats:{:.4f} path:{}".format(duration_extract, feats_save_path))

 all type paths below:
type1 dir:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_1_seg_224
type2 dir:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_2_seg_224
type3 dir:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224
type1 images number:5760
type2 images number:17384
type3 images number:9704
sample 1:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/100-resnet-152.npy
sample 2:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/100-resnet-200.npy
sample 3:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/100-resnext-50.npy
sample 4:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/100.jpg
sample 5:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_seg_224/1000-resnet-152.npy
sample 6:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_3_se

KeyboardInterrupt: 

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 22:56:56.212711. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


Extract test image features.

In [75]:
import os

def get_test_img_list(test_img_path):
    img_list = filter(lambda p: p[-4:] == ".jpg", os.listdir(test_img_path))
    return map(lambda img_name: "/".join([test_img_path, img_name]), img_list)

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 23:01:25.816570. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [77]:
###############################
# just for test               #
###############################

test_img_path = "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224"
test_img_list = get_test_img_list(test_img_path)
print len(test_img_list)
print test_img_list[:10]

512
['/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/0.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/1.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/10.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/100.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/101.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/102.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/103.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/104.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/105.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/106.jpg']


/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 23:01:34.570510. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [ ]:
import os
import mxnet as mx
from collections import namedtuple
import time, datetime
import cv2

################################################
# parameter initialization                     #
################################################

###############################
# extract feature on test set #
###############################

test_img_path = "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224"
img_path_list = get_test_img_list(test_img_path)
Batch = namedtuple('Batch', ['data'])
feats_save_path_list = map(lambda img_path: img_path.replace("jpg", "npy"), img_path_list)
model_path = "../model"
prefix_list = ["finetune-resnet-152-train-add-seg-224",\
               "finetune-resnet-200-train-add-seg-224",\
               "finetune-resnext-50-train-add-seg-224-lr-0.01"]
epoch_list = [3, 3, 3]
feats_save_prefix_list = ["resnet-152",\
                          "resnet-200",\
                          "resnext-50"]
synset_path="../model/synset.txt"
synsets = get_synset(synset_path)

################################################
# extract feature using each pretrained model  #
################################################

for m_idx in xrange(len(prefix_list)):
    prefix = "/".join([model_path, prefix_list[m_idx]])
    epoch = epoch_list[m_idx]
    feats_save_prefix = feats_save_prefix_list[m_idx]
    feature_extractor = get_feature_extractor(prefix, epoch)
    
    for img_idx in xrange(len(img_path_list)):
        time_stamp = datetime.datetime.now()
        print("{} {}th/{} img".format(time_stamp.strftime('%Y-%m-%d %H:%M:%S'), img_idx+1, len(img_path_list))),
        
        start_extract = time.time()
        
        img_path = img_path_list[img_idx]
        feats_save_path = feats_save_path_list[img_idx]
        img = get_image(img_path)

        feature_extractor.forward(Batch([mx.nd.array(img)]))
        feats = feature_extractor.get_outputs()[0].asnumpy()
        
        #####################
        # check features    #
        #####################
        #print feats.shape
        
        #####################
        # save features     #
        #####################
        feats_save_path = feats_save_path.replace(".", "-"+feats_save_prefix+".")
        np.save(feats_save_path, feats)
        
        finish_extract = time.time()
        duration_extract = finish_extract - start_extract
        
        print("calc and save feats:{:.4f} path:{}".format(duration_extract, feats_save_path))

 2017-06-09 23:02:27 1th/512 img calc and save feats:1.0504 path:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/0-resnet-152.npy
2017-06-09 23:02:28 2th/512 img calc and save feats:0.9712 path:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/1-resnet-152.npy
2017-06-09 23:02:29 3th/512 img calc and save feats:1.0169 path:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/10-resnet-152.npy
2017-06-09 23:02:30 4th/512 img calc and save feats:1.3943 path:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/100-resnet-152.npy
2017-06-09 23:02:32 5th/512 img calc and save feats:1.1143 path:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/101-resnet-152.npy
2017-06-09 23:02:33 6th/512 img calc and save feats:1.2884 path:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/102-resnet-152.npy
2017-06-09 23:02:34 7th/512 img calc and save feats:1.0203 path:/media/yuens/WIN10-ENTERTENMENT/Kaggle/I

# Step 2.5 Feature Concate

In [41]:
np1 = '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_1_seg_224/3070-resnet-152.npy'
np2 = '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_1_seg_224/3070-resnet-200.npy'
np3 = '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add_seg_224/Type_1_seg_224/3070-resnext-50.npy'

a = np.load(np1)
b = np.load(np2)
c = np.load(np3)

print a.shape
print b.shape
print c.shape

print type(a)
print type(b)
print type(c)

print np.concatenate((a,b,c), axis=0).shape
print np.concatenate((a,b,c), axis=1).shape

?np.concatenate

(1, 2048)
(1, 2048)
(1, 2048)
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
(3, 2048)
(1, 6144)


/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 21:40:15.140838. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [25]:
def make_submit(train_data_path, feature_path, sample_submission_path, submission_save_path):    
    # 训练分类器
    clf = train_xgboost(train_data_path, feature_path)
    
    # 取出测试集数据做预测
    df = pd.read_csv(sample_submission_path)
    x = np.array([np.mean(np.load(feature_path + '%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])
    pred = clf.predict(x)

    # 将结果写入文件准备提交
    df['cancer'] = pred
    df.to_csv(submission_save_path, index=False)
    print(df.head())

(1, 2048)


/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-08 21:46:48.573857. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-08 21:46:57.077648. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


SyntaxError: invalid syntax (<ipython-input-30-103a2f9e43e8>, line 1)

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-08 21:47:17.142431. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


'/home/yuens/Documents/charm/cervical-cancer-screening/code'

/usr/local/lib/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-06-09 22:58:55.048329. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
